<a href="https://colab.research.google.com/github/Nikoszav/MathForAI/blob/main/Ex2/CwEx2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANN with Numpy

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import random as rd
import _pickle as cPickle    
import gzip
import random
import pandas as pd



#Import dataset

In [ ]:
#load the file with the data 
#put the path of the file mnist.pkl.gz below
mnist_file = gzip.open('/content/drive/MyDrive/dataset/mnist.pkl.gz', 'rb') 
#load them and split them in training_data, validation_data, test_data
trainingData, validationData, testData = cPickle.load(mnist_file, encoding='latin1')

# One hot encoding

In [ ]:
#return a vector of 10 on and in the position of the number(class) is make it equal with 1(one hot encoding)
def yLabel(y):
    #initialize an array of length 10 with zero
    yLabel = np.zeros((10,1))
    
    #at y position it make it equals with 1
    yLabel[y] = 1.0
    
    return yLabel

#Preprocessing 

In [ ]:
#basic preprocsing to make our data in a way that our algorithm can undestand the data
trainingInputs = [np.reshape(x, (784,1)) for x in trainingData[0]]
trainingLabels = [yLabel(y) for y in trainingData[1]]

trainSet = zip(trainingInputs, trainingLabels)
    
validationInputs = [np.reshape(x, (784,1)) for x in validationData[0]]
validationSet = zip(validationInputs, validationData[1])

testInputs = [np.reshape(x, (784,1)) for x in testData[0]]
testSet = zip(testInputs, testData[1])


In [ ]:
trainSet = list(trainSet)
validationSet = list(validationSet)
testSet = list(testSet)

#Import functions 

In [ ]:
!python3 functions.py

In [ ]:
!pip install ipynb

In [ ]:
import ipynb.fs 
from .defs.functions import relu
from .defs.functions import reluDerivative
from .defs.functions import softmax
from .defs.functions import sigmoid
from .defs.functions import derivativeSigmoid

In [ ]:
#best softmax at the backprop 

#unitilization gausian distibution
'''
The classifier class:
in the constuctor we have the size of the nerwork, a list with the activation functions for each one of the hidden layers, the untis of the inputlayer(784),
a list with the hidden layers and the the numbers of units for each hidden layer, the units of the ouput layer(10)  
'''
class Classifier():
  #define the constractor for the classifier 
  #define layers, biases and weights
  def __init__(self, sizeNetwork, activationFunction, inputLayer, hiddenlayerList, outputLayer):
    #create a list which will contain the differents sizes og the layers of the network
    self.neuralNetworkParameters = [inputLayer]
    for hiddenLayer in hiddenlayerList:
      self.neuralNetworkParameters.append(hiddenLayer)

    self.neuralNetworkParameters.append(outputLayer)
    #define the number of layers the the neural network will have
    self.layers = sizeNetwork

    #initialize the biases of the neural network
    self.biases = [np.random.randn(y,1) for y in self.neuralNetworkParameters[1:]]
    #initilize the weights of the neural network
    self.weights = [np.random.randn(y,x) / np.sqrt(x) for x, y in zip(self.neuralNetworkParameters[:-1], self.neuralNetworkParameters[1:])]
    #define the activation function
    self.f = activationFunction
    #initliaze the cost function
    self.training_accuracy, self.training_cost, self.validationAcc = [], [], []
 
  #define the feedforward function 
  def feedforward(self, currentNeuron):
    i = 0
    for b, w in zip(self.biases, self.weights):     
      if i != len(hiddenLayerList):
        # print(f[i])

        linearFun = np.dot(w, currentNeuron)+b
        
        currentNeuron = self.activationFun(linearFun,self.f[i])

      
      else:
        currentNeuron = np.dot(w, currentNeuron)+b
        currentNeuron = np.exp(currentNeuron) / np.sum(np.exp(currentNeuron), axis=0)

    i+=1
    return currentNeuron

  # implematation of minibatch stochastic gradient descent
  def fit(self, trainSet, testSet, validationSet, learningRate, epochs, miniBatchSize):    
    
    trainSet = list(trainSet)
    n_train = len(trainSet)
    #speify the length of the trainset 
    lengthOfTrainData = len(trainSet)
    # lengthOfTestData = len(testSet)
    epoch =1
    continue_training = True
    #define and implement mini batch gradient descent 
   
    while continue_training:
      #shuffle data 
      random.shuffle(trainSet)
      
      miniBatches = [trainSet[y: y+miniBatchSize] for y in range(0, lengthOfTrainData, miniBatchSize)]
      for miniBatch in miniBatches:
        # need to update weights and biases in respect of the learning rate         
        self.updateParameters(miniBatch, learningRate, n_train)
      accuracy = self.accuracy(trainSet)
      print("Epoch: ", epoch , ": ", accuracy , " / ", lengthOfTrainData )
      self.training_accuracy.append(accuracy)
      self.training_cost.append(self.costentropy(trainSet))

      accuracy = (accuracy/len(trainSet)) * 100
      val_acc = self.evaluate(validationSet, valSet= True)
      self.validationAcc.append(val_acc)
      if val_acc > 95.0:
        continue_training = False
        print("Early finish after ", epoch, " epochs with training accuracy be ", accuracy)
      elif epoch >= epochs: 
        continue_training = False
        print("Finish after ", epoch, " epochs with training accuracy be ", accuracy)
      epoch += 1
  
  #function for updating parameters 
  def updateParameters(self,miniBatch, learningRate, n_train):
    #initliaze the lists which will store the weights and biases
    lmbda = 5.0
    listOfWeights = [np.zeros(weight.shape) for weight in self.weights] 
    listOfBiases = [np.zeros(bias.shape) for bias in self.biases] 

    for X, y in miniBatch:
      #the change of the weights and the biases after the backpropagation process 
      deltaListOfWeights, deltaListOfBiases = self.backpropagation(X, y)
      
      #update the list of biases
      listOfBiases = [b + dB for b, dB in zip(listOfBiases , deltaListOfBiases)]
      #update the list of weights
      listOfWeights = [w + dW for w, dW in zip(listOfWeights , deltaListOfWeights)]

    #updating the weights by using gradient descent
    # self.weights = [(1-learningRate*(lmbda/n_train)) * w - (learningRate/len(miniBatch)) * nw for w, nw in zip(self.weights, listOfWeights)]
    self.weights = [w - (learningRate/len(miniBatch)) * nw for w, nw in zip(self.weights, listOfWeights)]
    #updating the biases by using gradient descent
    self.biases = [b - (learningRate/len(miniBatch)) * nb for b, nb in zip(self.biases, listOfBiases)]

  

  def backpropagation(self, X, y):
    deltaW = [np.zeros(w.shape) for w in self.weights]
    deltaB = [np.zeros(b.shape) for b in self.biases]
    

    # implematation of feedfard process calculate linear model + activation function
    resultActivation = X

    #will keep all the results of the neurons layer by layer
    resultsActivation = [X]   
    
    #keep a list of z vector the output of a neuron 
    zs = []
    i = 0
    for b, w in zip(self.biases, self.weights):
    
      if i < len(hiddenLayerList):
        #calculating z vectors by using the theory
        #from theory we know that the output of a node is equal with the dot product of the weights and the previous nodes + the bias term      
        z = np.matmul(w, resultActivation) + b        
        #add z to the list 
        zs.append(z)
        # print(self.f[i])
        #apply the agtivation funnction that we have defined 
        resultActivation = self.activationFun(z, self.f[i])
        #store this result in our list
        resultsActivation.append(resultActivation)
      
      else:
        z = np.matmul(w, resultActivation) + b       
        #add z to the list 
        zs.append(z)
        #softmax for the last layer
        z = np.exp(z) / np.sum(np.exp(z), axis=0)
        resultsActivation.append(z)
      i += 1
      # Implematation of the backward procces 

    error = resultsActivation[-1] - y 

    # print(len(zs[-1]))
    deltaError = error * self.derivativeActivationFun(zs[-1], self.f[-1])
    # print(self.derivativeActivationFun(zs[-1]))
    deltaB[-1] = np.mean(deltaError)
    deltaW[-1] = np.dot(deltaError, resultsActivation[-2].transpose())
    
    for layer in range(2,self.layers):
      z = zs[-layer]
      deltaPred = self.derivativeActivationFun(z, self.f[layer-2])
      deltaError = np.matmul(self.weights[-layer + 1].transpose(), deltaError) * deltaPred
      deltaB[-layer] = np.mean(deltaError)
      deltaW[-layer] = np.dot(deltaError, resultsActivation[-layer - 1].transpose()) 

      
    return (deltaW, deltaB)


#Functions 

  #define the actuvation function that you choose
  def activationFun(self, x, f):
    if (f == "sigmoid"):
      return sigmoid(x)
    elif (f == "relu"):     
      # print("okay") 
      return relu(x)  
  
  #define the derivative of the actuvation function that you choose
  def derivativeActivationFun(self, x, f):
    if (f == "sigmoid"):
      return derivativeSigmoid(x)
    elif (f == "relu"):
      return reluDerivative(x)  
  

  #calculate the accuracy of the classifier on the test set
  def accuracy(self, data):
    accuracy = 0
    #create a list od tuples where first element is the predicted y and second element the actually output
    results = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x,y) in data]
    acc = sum(int(x == y) for (x,y) in results)
    accuracy = acc/len(data) * 100
    print('Accuracy on Training data set: ', accuracy)
    return accuracy


  #method to evaluate the classifier 
  def evaluate(self, data, valSet):
    if valSet:
      #create a list od tuples where first element is the predicted y and second element the actually output
      results = [(np.argmax(self.feedforward(x)), y) for (x,y) in data]
      accuracy = sum(int(x == y) for (x,y) in results)
      accuracy = accuracy/len(data) * 100
      print('Accuracy on validatation data set: ', accuracy)
      return accuracy
    else:
      #create a list od tuples where first element is the predicted y and second element the actually output
      results = [(np.argmax(self.feedforward(x)), y) for (x,y) in data]
      accuracy = sum(int(x == y) for (x,y) in results)
      
      accuracy = accuracy/len(data) * 100
      print('Accuracy on test data set: ', accuracy)


  #calculate the cost based on the costentropy function 
  def costentropy(self, data):
    cost = 0.0
    for x, y in data:
        y_pred = self.feedforward(x)
        y_pred = softmax(y_pred)
        
        # print(y_pred, y)
        # y_pred
        cost = np.mean(-y * np.log(y_pred)- (1.0-y) * np.log(1.0 - y_pred))
    
    print(cost)
    return cost

  def matrices(self):
    return self.training_accuracy, self.training_cost, self.validationAcc   
  

In [ ]:
#specify the number of units for each of the hidden layer
#if you want more than one layers you can fo smt like this hiddenLayerList = [1000, 100, 10] 3 hidden layer and the first has 1000units, second 100, third 10
hiddenLayerList = [1000]
#the activation function the size of this list must map the hidden layer list 
activationFun = ["sigmoid"]
#call the class
net = Classifier(3, activationFun, 784, hiddenLayerList,10)
#call the fit method 
net.fit(trainSet, testSet, validationSet, learningRate= 0.01, epochs = 30, miniBatchSize= 10)
#return the accuracies lists
trainingAcc, trainingLoss, validationAcc  = net.matrices()
#evaluate on the test set
net.evaluate(testSet, valSet= False)

#create dictionary which store the accuracies and the costs

In [ ]:
evaluation = {}
evaluation["trainAccSigmoid"] = trainingAcc
evaluation["trainAccRelu"] = trainingAcc1

evaluation["trainLossSigmoid"] = trainingLoss
evaluation["trainLossRelu"] = trainingLoss1

evaluation["validationAccSigmoid"] = validationAcc
evaluation["validationAccRelu"] = validationAcc1

#Plot graphs

In [ ]:
plt.plot(evaluation["trainAccSigmoid"], label='sigmoid')
plt.plot(evaluation["trainAccRelu"], label='relu')
plt.ylabel('Training Accuracy')
plt.xlabel('Epochs')
plt.xticks(np.arange(0,32,2))
plt.title('Training Accuracy per epoch')
plt.legend()

In [ ]:
plt.plot(evaluation["trainLossSigmoid"], label='sigmoid')
plt.plot(evaluation["trainLossRelu"], label='relu')
plt.ylabel('Training loss')
plt.xlabel('Epochs')
plt.xticks(np.arange(0,32,2))
plt.title('Training loss per epoch')
plt.legend()

In [ ]:
plt.plot(evaluation["validationAccSigmoid"], label = 'sigmoid')
plt.plot(evaluation["validationAccRelu"], label = 'relu') 
plt.ylabel('validation accuracy')
plt.xticks(np.arange(0,32,2))
plt.xlabel('Epochs')   
plt.title('Validation accuracy per epoch')
plt.legend()